# Prediction with TropiGAT : 
### I- Prepare the model
### II- Run the predictions on matrices
### III- Run the predictions on experimentally validated depolymerases
***

### Prepare the model: 

In [2]:
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero , HeteroConv , GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score , matthews_corrcoef

import TropiGAT_functions 
#from TropiGAT_functions import get_top_n_kltypes ,clean_print 

import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
import logging
from multiprocessing.pool import ThreadPool
warnings.filterwarnings("ignore")

# *****************************************************************************
# Load the Dataframes :
path_benchmark = "/media/concha-eloko/Linux/PPT_clean/benchmark"

path_work = "/media/concha-eloko/Linux/PPT_clean/reviewed_models/best_models"

# Load best parameters : 
with open(f"/media/concha-eloko/Linux/PPT_clean/trainer_best_parameters/DAG_models_best_para.json", "r") as f:
    best_parameters = json.load(f)

/media/concha-eloko/Linux/conda_envs/torch_geometric/lib/python3.11/site-packages/torch_geometric/typing.py:18: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /media/concha-eloko/Linux/conda_envs/torch_geometric/lib/python3.11/site-packages/libpyg.so: undefined symbol: _ZN3c1010Dispatcher17runRecordFunctionERN2at14RecordFunctionESt17reference_wrapperIKNS_14FunctionSchemaEENS_11DispatchKeyE
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/media/concha-eloko/Linux/conda_envs/torch_geometric/lib/python3.11/site-packages/torch_geometric/typing.py:31: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /media/concha-eloko/Linux/conda_envs/torch_geometric/lib/python3.11/site-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/media/concha-eloko/Linux/conda_envs/torch_geometric/lib

> TropiSAGE regular:

In [3]:
path_ensemble = f"{path_work}/best_models_TropiSAGE"

dico_models, errors = TropiGAT_functions.make_ensemble_TropiSAGE_r(path_ensemble)

> TropiSAGE ultrafiltration: 

In [12]:
path_ensemble = f"{path_work}/best_models_TropiSAGE_UF"

dico_models, errors = TropiGAT_functions.make_ensemble_TropiSAGE_r(path_ensemble)

***
### Run predictions on matrices: 

In [4]:
# ***************************************************************************
# Ferriol inferences : 
path_project = "/media/concha-eloko/Linux/77_strains_phage_project"
path_Dpo_domain_org = "/media/concha-eloko/Linux/depolymerase_building/clean_77_phages_depo"

dpo_embeddings = pd.read_csv(f"{path_project}/rbp_work/Dpo_domains_77.esm2.embedding.2406.csv", sep = "," , header = None, index_col = 0)


# ***************************************************************************
# Beamud inferences : 
path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

bea_embeddings = pd.read_csv(f"{path_project}/Bea_phages.esm2.embedding.csv", sep = "," , header = None)
bea_embeddings = bea_embeddings.drop([1281] , axis = 1)
bea_embeddings.set_index([0], inplace = True)


# ***************************************************************************
# Towndsend inferences : 
path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

towndsend_embeddings = pd.read_csv(f"{path_project}/Townsed_phages.esm2.embedding.1112.csv", sep = "," , header = None)
towndsend_embeddings = towndsend_embeddings.drop([1281] , axis = 1)
towndsend_embeddings.set_index([0], inplace = True)


> Run the predictions :

In [13]:
# Run the predictions ferriol :
ferriol_predictions = {}
for dpo in tqdm(dpo_embeddings.index) : 
    try : 
        graph_dpo = TropiGAT_functions.make_query_graph([dpo_embeddings.loc[dpo].values])
        pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
        ferriol_predictions[dpo] = pred
    except Exception as e :
        print(dpo, e)
        
# Run the predictions Bea :
bea_predictions = {}
for dpo in tqdm(bea_embeddings.index) : 
    graph_dpo = TropiGAT_functions.make_query_graph([bea_embeddings.loc[dpo].values])
    pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
    bea_predictions[dpo] = pred
    
# Run the predictions towndsend :
towndsend_predictions = {}
for dpo in tqdm(towndsend_embeddings.index) : 
    graph_dpo = TropiGAT_functions.make_query_graph([towndsend_embeddings.loc[dpo].values])
    pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
    towndsend_predictions[dpo] = pred

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:03<00:00, 14.60it/s]


> Write the file:

In [14]:
for prot in ferriol_predictions:
    if prot == "K42PH8_Dpo_domain": 
        ferriol_predictions["K42PH8__cds_48_Dpo_domain"] = ferriol_predictions[prot]
        del ferriol_predictions[prot]
        break
        
        
predictions = [ferriol_predictions , bea_predictions , towndsend_predictions]

        
with open(f"{path_benchmark}/TropiSAGE_UF.review.matrices.tsv", "w") as outfile:
    for prediction in predictions:
        for prot in prediction:
            if prediction[prot] == "No hits" or len(prediction[prot]) == 0:
                outfile.write(f"{prot}\tNo hits\n")
            else:
                outfile.write(f"{prot}\t")
                hits = [f"{kltype}:{round(score, 3)}" for kltype, score in prediction[prot].items()]
                sorted_hits = " ; ".join(sorted(hits, key=lambda x: float(x.split(":")[1]), reverse=True))
                outfile.write(sorted_hits)
                outfile.write("\n")

In [7]:
len(ferriol_predictions)

142

***
### Work on experimentally validated depolymerases:

In [8]:
path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

others_embeddings = pd.read_csv(f"{path_project}/Others_all.esm2.embedding.csv", sep = "," , header = None)
others_embeddings.set_index([0], inplace = True)

In [15]:
# Run the predictions Others :
other_predictions = {}
for index, dpo in tqdm(enumerate(others_embeddings.index)) :
    if dpo not in ["MN781108.1_prot_QGZ15323.1_262"] :
        try : 
            graph_dpo = TropiGAT_functions.make_query_graph([others_embeddings.loc[dpo].values])
            pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
            other_predictions[dpo] = pred
        except Exception as e :
            print(e, dpo)

22it [00:01, 16.01it/s]

running_mean should contain 1 elements not 1280 ON146449.1_prot_UPW35150.1_13


32it [00:02, 14.90it/s]

running_mean should contain 1 elements not 1280 ON146449.1_prot_UPW35138.1_1


56it [00:04, 12.65it/s]

running_mean should contain 1 elements not 1280 ON146449.1_prot_UPW35150.1_13
running_mean should contain 1 elements not 1280 ON146449.1_prot_UPW35138.1_1


In [16]:
predictions = [other_predictions]

with open(f"{path_benchmark}/TropiSAGE_UF.review.exp_val_depolymerase.tsv", "w") as outfile:
    for prediction in predictions:
        prediction_sorted = dict(sorted(prediction.items()))
        for prot in prediction_sorted:
            if prediction_sorted[prot] == "No hits" or len(prediction_sorted[prot]) == 0:
                outfile.write(f"{prot}\tNo hits\n")
            else:
                outfile.write(f"{prot}\t")
                hits = [f"{kltype}:{round(score, 3)}" for kltype, score in prediction_sorted[prot].items()]
                sorted_hits = " ; ".join(sorted(hits, key=lambda x: float(x.split(":")[1]), reverse=True))
                outfile.write(sorted_hits)
                outfile.write("\n")